Import libraries

In [40]:
import mujoco
import mujoco.viewer
import numpy as np

Load XML file

In [ ]:
#load model
model = mujoco.MjModel.from_xml_path(r"F:\CSE499\mjctrl-main-n\Jetarm\jetarmnew.xml")
data = mujoco.MjData(model)

# Print model info 
print("Model loaded ✅")
print("nq (generalized positions):", model.nq)
print("nv (generalized velocities):", model.nv)
print("nu (number of actuators):", model.nu)
print("nbody (number of bodies):", model.nbody)
print("nsite (number of sites):", model.nsite)
print("nmocap (mocap bodies):", model.nmocap)




Model loaded ✅
nq (generalized positions): 11
nv (generalized velocities): 11
nu (number of actuators): 6
nbody (number of bodies): 15
nsite (number of sites): 2
nmocap (mocap bodies): 0


In [42]:
# Print joint info
for i in range(model.njnt):
    print(f"Joint {i}: name={mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_JOINT, i)}, type={model.jnt_type[i]}")

# Print actuator mappings and joint properties
print("Actuator mappings:", model.actuator_trnid)
try:
    print("Joint damping:", model.dof_damping)  # Use dof_damping for compatibility
    print("Joint ranges:", model.jnt_range)
except AttributeError:
    print("⚠️ Joint damping or ranges not accessible in this MuJoCo version.")

Joint 0: name=joint1, type=3
Joint 1: name=joint2, type=3
Joint 2: name=joint3, type=3
Joint 3: name=joint4, type=3
Joint 4: name=joint5, type=3
Joint 5: name=r_joint, type=3
Joint 6: name=l_joint, type=3
Joint 7: name=r_in_joint, type=3
Joint 8: name=r_out_joint, type=3
Joint 9: name=l_in_joint, type=3
Joint 10: name=l_out_joint, type=3
Actuator mappings: [[ 0 -1]
 [ 1 -1]
 [ 2 -1]
 [ 3 -1]
 [ 4 -1]
 [ 5 -1]]
Joint damping: [0.01  0.01  0.01  0.01  0.001 0.    0.    0.    0.    0.    0.   ]
Joint ranges: [[-2.1   2.1 ]
 [-2.1   2.1 ]
 [-2.1   2.1 ]
 [-2.1   2.1 ]
 [-2.1   2.1 ]
 [ 0.    1.57]
 [ 0.    1.57]
 [-1.57  0.  ]
 [ 0.    1.57]
 [-1.57  0.  ]
 [ 0.    1.57]]


In [43]:
# --- Confirm if mocap body exists ---
if model.nmocap > 0:
    print("Mocap body IDs:", range(model.nmocap))
    print("Mocap pos before sim:", data.mocap_pos.copy())
    print("Mocap quat before sim:", data.mocap_quat.copy())
else:
    print("⚠️ No mocap body defined in XML!")

⚠️ No mocap body defined in XML!


In [44]:
# --- Gravity compensation ---
def compute_gravity_compensation(model, data):
    mujoco.mj_resetData(model, data)
    mujoco.mj_forward(model, data)  # Compute forward dynamics
    gravity_torques = data.qfrc_bias[:5].copy()  # Gravity torques for arm joints
    return gravity_torques

In [ ]:

# --- Run simulation with viewer ---
with mujoco.viewer.launch_passive(model, data) as viewer:
    step = 0
    kp = 50.0  # proportional gain
    kd = 20.0  # derivative gain
    desired_qpos = np.zeros(model.nq)  # Target: hold initial positions
    gravity_torques = compute_gravity_compensation(model, data)
    print("Gravity torques:", gravity_torques.round(6))  # Debug gravity torques
    
    while viewer.is_running() and step < 5000:
        if model.nu > 0:
            # PD controller with gravity compensation
            pd_control = -kp * (data.qpos[:5] - desired_qpos[:5]) - kd * data.qvel[:5]
            data.ctrl[:5] = pd_control + gravity_torques
            data.ctrl[5] = 0.0  # Keep gripper stationary
            # Clamp control inputs
            for i in range(5):
                data.ctrl[i] = np.clip(data.ctrl[i], -2.1, 2.1)
            data.ctrl[5] = np.clip(data.ctrl[5], -1.5, 1.5)
        
        mujoco.mj_step(model, data)
        if step % 100 == 0:
            print(f"Step {step}")
            print(" qpos:", data.qpos[:].round(6))
            print(" qvel:", data.qvel[:].round(6))
            print(" ctrl:", data.ctrl[:].round(6))
            print(" gravity torques (dynamic):", data.qfrc_bias[:5].round(6))
            if model.nmocap > 0:
                print(" mocap_pos:", data.mocap_pos.round(6))
                print(" mocap_quat:", data.mocap_quat.round(6))
        step += 1

Gravity torques: [0.       0.017221 0.017221 0.017221 0.      ]
Step 0
 qpos: [ 0.  0. -0. -0.  0.  0.  0. -0. -0. -0. -0.]
 qvel: [ 0.  0. -0. -0.  0.  0.  0. -0. -0. -0. -0.]
 ctrl: [0.       0.017221 0.017221 0.017221 0.       0.      ]
 gravity torques (dynamic): [0.       0.017221 0.017221 0.017221 0.      ]
Step 100
 qpos: [-0.  0. -0. -0. -0.  0.  0. -0.  0. -0.  0.]
 qvel: [-0.  0. -0. -0. -0.  0.  0. -0.  0. -0.  0.]
 ctrl: [0.       0.017221 0.017221 0.017221 0.       0.      ]
 gravity torques (dynamic): [ 0.        0.017221  0.017221  0.017221 -0.      ]
Step 200
 qpos: [-0.  0. -0. -0. -0.  0.  0. -0.  0. -0.  0.]
 qvel: [-0.  0. -0. -0. -0.  0.  0. -0.  0. -0.  0.]
 ctrl: [0.       0.017221 0.017221 0.017221 0.       0.      ]
 gravity torques (dynamic): [ 0.        0.017221  0.017221  0.017221 -0.      ]
Step 300
 qpos: [-0.  0. -0. -0. -0.  0.  0. -0.  0. -0.  0.]
 qvel: [-0.  0. -0. -0. -0.  0.  0. -0.  0. -0.  0.]
 ctrl: [0.       0.017221 0.017221 0.017221 0.       0